In [87]:
from conexionbd import DAO


# Datos de tabla modelos GD para extraccion part number
query_final_partnumber = """
SELECT 
	SN.DS_PartNumber PARTNUMBER, 
	SN.DS_IMPORTADOR,
    SN.DS_LIBR_TRIBU
FROM dw.SN_DataSunat SN
WHERE SN.DS_LIBR_TRIBU IN (
                                        SELECT RUC
                                        FROM
                                        (
                                        SELECT DS_LIBR_TRIBU RUC,
                                                COUNT(*) IMPORTACIONES
                                        FROM dw.SN_DataSunat
                                        WHERE DS_LIBR_TRIBU <> 'No Disponi.'
                                        GROUP BY DS_IMPORTADOR
                                        ORDER BY 2 DESC
                                        LIMIT 30
                                        ) IMPORTADORES
                        )
# AND DS_PartNumber = '82RK00WCLM'                        
AND DS_FECHA >= '2024-01-01' AND  DS_FECHA < '2024-12-01' 
AND DS_PARTNUMBER IS NOT NULL
GROUP BY SN.DS_PARTNUMBER, SN.DS_IMPORTADOR                
"""

columns_partnumber = ['PART_NUMBER', 'IMPORTADOR','RUC']


data = DAO().carga_data_sql(query_final_partnumber, columns_partnumber)
data['IMPORTADOR'] = data['IMPORTADOR'].str.strip().str.upper()
data


Conexión exitosa a la base de datos en 0.00438237190246582 segundos
Se obtuvieron 5257 datos en 2.519430160522461 segundos


,PART_NUMBER,IMPORTADOR,RUC
0,001R00610,COMPUDISKETT S R L,20123053037
1,001R00613,COMPUDISKETT S R L,20123053037
2,001R00613,MAXIMA INTERNACIONAL S.A.,20127745910
3,001R00613,XEROX DEL PERU S A,20100119065
4,006R01160,COMPUDISKETT S R L,20123053037
...,...,...,...
5252,ZACH0149PE,INGRAM MICRO S.A.C.,20267163228
5253,ZADA0354PE,GRUPO DELTRON S.A.,20212331377
5254,ZADA0354PE,INGRAM MICRO S.A.C.,20267163228
5255,ZADB0033PE,GRUPO DELTRON S.A.,20212331377


In [88]:
lista_palabras_retirar = ["@codprodc-", "@tarjeta-", "@modelo-", "@simil_prox-"]

def withdraw_words(x):
    if x is not None and x.startswith("@"):
        return x[x.find("-")+1:]
    else:
        return x

data['PART_NUMBER'] = data['PART_NUMBER'].apply(withdraw_words)

# data.to_excel('palabras_retiradas.xlsx', index=False)

In [89]:
data.head(100)

,PART_NUMBER,IMPORTADOR,RUC
0,001R00610,COMPUDISKETT S R L,20123053037
1,001R00613,COMPUDISKETT S R L,20123053037
2,001R00613,MAXIMA INTERNACIONAL S.A.,20127745910
3,001R00613,XEROX DEL PERU S A,20100119065
4,006R01160,COMPUDISKETT S R L,20123053037
...,...,...,...
95,04071-02620000,GRUPO DELTRON S.A.,20212331377
96,04071-02620100,GRUPO DELTRON S.A.,20212331377
97,04071-02730000,GRUPO DELTRON S.A.,20212331377
98,04071-02730100,GRUPO DELTRON S.A.,20212331377


In [90]:
importadores = data['IMPORTADOR'].unique()


dict_importadores = {}
for importador in importadores:
    list_partnumber = data['PART_NUMBER'][(data['IMPORTADOR']==importador) & ~(data['PART_NUMBER'].isnull())].values
    dict_importadores[importador] = list_partnumber

print(dict_importadores)

{'COMPUDISKETT S R L': array(['001R00610', '001R00613', '006R01160', '006R01518', '006R01520',
       '008R13061', '013R00662', '013R00670', '0322C001AA', '0351C001AA',
       '0352C001AA', '0353C001AA', '0354C001AA', '0376C001AA',
       '0405C001AA', '0406C001AA', '0407C001AA', '0408C001AA',
       '0615B050AA', '0617B050AA', '0692C004AA', '0692C005AA',
       '100-100000065BOX', '100-100000147BOX', '100-100000252BOX',
       '100-100000263BOX', '100-100000457BOX', '100-100000514WOF',
       '100-100000590BOX', '100-100000591WOF', '100-100000593WOF',
       '100-100000910WOF', '100-100000926WOF', '100-100000931BOX',
       '100-100001236BOX', '100-100001237BOX', '100-100001405WOF',
       '100-100001488BOX', '100-100001590BOX', '101R00474', '106R01305',
       '106R01571', '106R01572', '106R01573', '106R02236', '106R02306',
       '106R02310', '106R02312', '106R02721', '106R02741', '106R02752',
       '106R02755', '106R02760', '106R02761', '106R02762', '106R02763',
       '106R02773'

In [ ]:

from matplotlib import axis


df_deltron = data[(data['IMPORTADOR']=='GRUPO DELTRON S.A.') & ~(data['PART_NUMBER'].isnull())]


def comparison_partnumber(serie, list_partnumber):
    
    for partnumber in list_partnumber:
       if serie in partnumber:
           return 1
    return 0


for importador in importadores:
    if importador in dict_importadores.keys():
        df_deltron[f"part_number_{importador}"] = df_deltron['PART_NUMBER'].apply(comparison_partnumber, args=(dict_importadores.get(importador),))
    


df_deltron = df_deltron.drop_duplicates(subset=['PART_NUMBER'], keep='first')


df_deltron.to_excel('partnumber_deltron.xlsx', index=False)
df_filtrado = df_deltron[df_deltron['PART_NUMBER'] == '82RK00WCLM']
df_filtrado.head()




C:\Users\gustavo.grillo\AppData\Local\Temp\ipykernel_14876\178592320.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deltron[f"part_number_{importador}"] = df_deltron['PART_NUMBER'].apply(comparison_partnumber, args=(dict_importadores.get(importador),))
C:\Users\gustavo.grillo\AppData\Local\Temp\ipykernel_14876\178592320.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deltron[f"part_number_{importador}"] = df_deltron['PART_NUMBER'].apply(comparison_partnumber, args=(dict_importadores.get(imp

,PART_NUMBER,IMPORTADOR,RUC,part_number_COMPUDISKETT S R L,part_number_MAXIMA INTERNACIONAL S.A.,part_number_XEROX DEL PERU S A,part_number_GRUPO DELTRON S.A.,part_number_DATACONT S.A.C.,part_number_ADU SYSTEMS S.R.L.,part_number_PRODUCTIVE BUSINESS SOLUTIONS GROUP PERU,...,part_number_IBM DEL PERU S A C,part_number_HEWLETT - PACKARD PERU S.R.L.,part_number_EXPORTADORA IMPORTADORA IGARASHI ASCENCI,part_number_EPSON PERU S.A,part_number_CORPORACION ANDES PRODUCTS SOCIEDAD ANON,part_number_SAMSUNG ELECTRONICS PERU SAC,part_number_HUAWEI DEL PERU SAC,part_number_GLOBAL MARKET IMPORT SOCIEDAD ANONIMA CE,part_number_DELL PERU S.A.C,part_number_LEXMARK INTERNATIONAL DE PERU S.R.L.
1593,82RK00WCLM,GRUPO DELTRON S.A.,20212331377,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
